In [1]:
import pandas as pd 
# from sodapy import Socrata  # API for data

In [103]:
# client = Socrata("data.cincinnati-oh.gov", None)
# crash_raw = pd.DataFrame.from_records(
#                 client.get('rvmt-pkmq', limit=1000000)
#             )

# Using the API is faster than using pd.read_json

# Crash Data Exploration

In [106]:
crash_raw.shape

(324723, 28)

In [107]:
crash_raw.head()  # What does the table look like?

,address_x,latitude_x,longitude_x,community_council_neighborhood,cpd_neighborhood,sna_neighborhood,crashdate,crashseverity,crashseverityid,datecrashreported,...,roadcontour,roadsurface,roadclass,roadclassdesc,unittype,typeofperson,weather,age,gender,crashlocation
0,36XX READING RD,39.148761,-84.512611,CLIFTON,CLIFTON,CLIFTON,2023-02-08T23:04:17.000,5 - PROPERTY DAMAGE ONLY,201905,2023-02-08T23:02:41.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,47XX READING RD,39.171042,-84.469211,BOND HILL,BONDHILL,BOND HILL,2023-02-08T22:30:00.000,4 - INJURY POSSIBLE,201904,2023-02-08T22:30:00.000,...,1 - STRAIGHT LEVEL,"2 - BLACKTOP, BITUMINOUS, ASPHALT",2,Major Arterial Roads,99 - UNKNOWN OR HIT/SKIP,D - DRIVER,4 - RAIN,NaN,NaN,NaN
2,47XX READING RD,39.170372,-84.468331,BOND HILL,BONDHILL,BOND HILL,2023-02-08T22:30:00.000,4 - INJURY POSSIBLE,201904,2023-02-08T22:30:00.000,...,1 - STRAIGHT LEVEL,"2 - BLACKTOP, BITUMINOUS, ASPHALT",2,Major Arterial Roads,03 - SPORT UTILITY VEHICLE,D - DRIVER,4 - RAIN,20,F - FEMALE,NaN
3,4XX W KING DR,39.135472,-84.501351,CORRYVILLE,CORRYVILLE,CORRYVILLE,2023-02-08T22:10:00.000,5 - PROPERTY DAMAGE ONLY,201905,2023-02-08T22:12:00.000,...,3 - CURVE LEVEL,"2 - BLACKTOP, BITUMINOUS, ASPHALT",2,Major Arterial Roads,01 - PASSENGER CAR,D - DRIVER,4 - RAIN,18,M - MALE,NaN
4,15XX HARRISON AV,39.125139,-84.549324,SOUTH FAIRMOUNT,SOUTH FAIRMOUNT,SOUTH FAIRMOUNT,2023-02-08T21:30:00.000,5 - PROPERTY DAMAGE ONLY,201905,2023-02-08T21:40:00.000,...,1 - STRAIGHT LEVEL,"2 - BLACKTOP, BITUMINOUS, ASPHALT",NaN,NaN,01 - PASSENGER CAR,D - DRIVER,4 - RAIN,42,M - MALE,NaN


In [108]:
crash_raw.dtypes  # Columns and data types

address_x                         object
latitude_x                        object
longitude_x                       object
community_council_neighborhood    object
cpd_neighborhood                  object
sna_neighborhood                  object
crashdate                         object
crashseverity                     object
crashseverityid                   object
datecrashreported                 object
dayofweek                         object
instanceid                        object
localreportno                     object
zip                               object
injuries                          object
lightconditionsprimary            object
mannerofcrash                     object
roadconditionsprimary             object
roadcontour                       object
roadsurface                       object
roadclass                         object
roadclassdesc                     object
unittype                          object
typeofperson                      object
weather         

## Reformatting Data

In [88]:
crash_data = crash_raw 

In [89]:
# Converting strings to dates
crash_data.loc[:, 'crashdate'] = pd.to_datetime(crash_data.crashdate)
crash_data.loc[:, 'datecrashreported'] = pd.to_datetime(crash_data.datecrashreported)

C:\Users\Ern\AppData\Local\Temp\ipykernel_8432\1890883014.py:2: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  crash_data.loc[:, 'crashdate'] = pd.to_datetime(crash_data.crashdate)
C:\Users\Ern\AppData\Local\Temp\ipykernel_8432\1890883014.py:3: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  crash_data.loc[:, 'datecrashreported'] = pd.to_datetime(crash_data.datecrashreported)


In [90]:
# Renaming to snake_case 
crash_data.rename(columns={'crashdate':'crash_date', 
                           'crashseverity':'crash_severity', 
                           'crashseverityid':'crash_severity_id',
                           'datecrashreported':'date_crash_reported',
                           'dayofweek':'day_of_week',
                           'instanceid':'instance_id',
                           'localreportno':'local_report_no',
                           'lightconditionsprimary':'light_condits_prim',
                           'mannerofcrash':'manner_of_crash',
                           'roadconditionsprimary':'rd_condits_prim',
                           'roadcontour':'rd_contour',
                           'roadsurface':'rd_surface',
                           'roadclass':'rd_class',
                           'roadclassdesc':'rd_class_descr',
                           'unittype':'unit_type',
                           'typeofperson':'person_type'}, 
                        inplace=True)

In [91]:
# Converting float zip codes to zip codes 
crash_data.loc[:,'zip'] = crash_data.zip.astype(str)
crash_data.loc[:,'zip'] = crash_data.zip.str[:5]

In [92]:
crash_data.dtypes   # Check that conversions worked

address_x                                 object
latitude_x                               float64
longitude_x                              float64
community_council_neighborhood            object
cpd_neighborhood                          object
sna_neighborhood                          object
crash_date                        datetime64[ns]
crash_severity                            object
crash_severity_id                        float64
date_crash_reported               datetime64[ns]
day_of_week                               object
instance_id                               object
local_report_no                            int64
zip                                       object
injuries                                  object
light_condits_prim                        object
manner_of_crash                           object
rd_condits_prim                           object
rd_contour                                object
rd_surface                                object
rd_class            

In [93]:
crash_raw.describe()

,latitude_x,longitude_x,crash_severity_id,local_report_no,rd_class,age
count,3.246970e+05,3.246950e+05,324713.000000,3.247230e+05,200951.000000,284662.000000
mean,2.663970e+03,-1.213147e+04,81324.190106,1.840774e+08,2.824748,37.470249
std,1.478429e+06,6.859273e+06,99024.233885,7.724166e+08,1.484080,16.495234
min,-8.449010e+01,-3.908552e+09,1.000000,1.400000e+01,0.000000,0.000000
25%,3.911737e+01,-8.454949e+01,3.000000,1.550166e+08,2.000000,24.000000
50%,3.913662e+01,-8.451499e+01,3.000000,1.850018e+08,2.000000,34.000000
75%,3.916096e+01,-8.448473e+01,201905.000000,2.050078e+08,4.000000,49.000000
max,8.424329e+08,9.800000e-04,201905.000000,2.201140e+11,10.000000,123.000000


In [94]:
crash_data.address_x    # Will want to separate block # from street.

0                36XX READING RD
1                47XX READING RD
2                47XX READING RD
3                 4XX W  KING DR
4               15XX HARRISON AV
                   ...          
324718    30XX WESTWOOD NORTHERN
324719    30XX WESTWOOD NORTHERN
324720              1XX E 9TH ST
324721              1XX E 9TH ST
324722            8XX NORTH BEND
Name: address_x, Length: 324723, dtype: object

In [95]:
crash_data.head()

,address_x,latitude_x,longitude_x,community_council_neighborhood,cpd_neighborhood,sna_neighborhood,crash_date,crash_severity,crash_severity_id,date_crash_reported,...,rd_contour,rd_surface,rd_class,rd_class_descr,unit_type,person_type,weather,age,gender,crashlocation
0,36XX READING RD,39.148761,-84.512611,CLIFTON,CLIFTON,CLIFTON,2023-02-08 23:04:17,5 - PROPERTY DAMAGE ONLY,201905.0,2023-02-08 23:02:41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,47XX READING RD,39.171042,-84.469211,BOND HILL,BONDHILL,BOND HILL,2023-02-08 22:30:00,4 - INJURY POSSIBLE,201904.0,2023-02-08 22:30:00,...,1 - STRAIGHT LEVEL,"2 - BLACKTOP, BITUMINOUS, ASPHALT",2.0,Major Arterial Roads,99 - UNKNOWN OR HIT/SKIP,D - DRIVER,4 - RAIN,NaN,NaN,NaN
2,47XX READING RD,39.170372,-84.468331,BOND HILL,BONDHILL,BOND HILL,2023-02-08 22:30:00,4 - INJURY POSSIBLE,201904.0,2023-02-08 22:30:00,...,1 - STRAIGHT LEVEL,"2 - BLACKTOP, BITUMINOUS, ASPHALT",2.0,Major Arterial Roads,03 - SPORT UTILITY VEHICLE,D - DRIVER,4 - RAIN,20.0,F - FEMALE,NaN
3,4XX W KING DR,39.135472,-84.501351,CORRYVILLE,CORRYVILLE,CORRYVILLE,2023-02-08 22:10:00,5 - PROPERTY DAMAGE ONLY,201905.0,2023-02-08 22:12:00,...,3 - CURVE LEVEL,"2 - BLACKTOP, BITUMINOUS, ASPHALT",2.0,Major Arterial Roads,01 - PASSENGER CAR,D - DRIVER,4 - RAIN,18.0,M - MALE,NaN
4,15XX HARRISON AV,39.125139,-84.549324,SOUTH FAIRMOUNT,SOUTH FAIRMOUNT,SOUTH FAIRMOUNT,2023-02-08 21:30:00,5 - PROPERTY DAMAGE ONLY,201905.0,2023-02-08 21:40:00,...,1 - STRAIGHT LEVEL,"2 - BLACKTOP, BITUMINOUS, ASPHALT",NaN,NaN,01 - PASSENGER CAR,D - DRIVER,4 - RAIN,42.0,M - MALE,NaN


In [96]:
crash_data.rd_contour.unique()

array([nan, '1 - STRAIGHT LEVEL', '3 - CURVE LEVEL', '4 - CURVE GRADE',
       '2 - STRAIGHT GRADE', '9 - UNKNOWN'], dtype=object)

In [97]:
# Separate scores from descriptions
crash_data[['crash_severity','crash_severity_descr']] = crash_data.crash_severity.str.split(pat = ' - ', expand=True)
crash_data[['rd_contour','rd_contour_descr']] = crash_data.rd_contour.str.split(pat = ' - ', expand=True)
crash_data.rd_contour = crash_data.rd_contour.astype('Int64')

In [98]:
crash_data.dtypes

address_x                                 object
latitude_x                               float64
longitude_x                              float64
community_council_neighborhood            object
cpd_neighborhood                          object
sna_neighborhood                          object
crash_date                        datetime64[ns]
crash_severity                            object
crash_severity_id                        float64
date_crash_reported               datetime64[ns]
day_of_week                               object
instance_id                               object
local_report_no                            int64
zip                                       object
injuries                                  object
light_condits_prim                        object
manner_of_crash                           object
rd_condits_prim                           object
rd_contour                                 Int64
rd_surface                                object
rd_class            

In [99]:
crash_data.shape

(324723, 30)

In [100]:
len(crash_data.instance_id.unique())

167783

Since the size of the table is larger than the number of unique IDs, we know that one accident may be captured in several rows.